In [1]:
import numpy as np
import math

In [2]:
class ActivationFunction:

    def __init__(self, func, leaky_val=0.01):
        self.func = func
        self.leaky_val = leaky_val

    def _linear(self, x):
        return x

    def _sigmoid(self, x):
        return (1/(1+ (math.e **(-x))))
        
    def _tanh(self, x):
        return (2 *  self._sigmoid(2*x) -1)
        
    def _reLU(self, x):
        return max(0,x)

    def _leakyReLU(self, x):
        if(x > 0):
            return x
        return self.leaky_val*x


    def gradient(self, x):
        if self.func == 'linear':
            return 1
        elif self.func == 'sig':
            calc = self._sigmoid(x)
            return (calc * (1 - calc))
        elif self.func == 'tanh':
            return 1 - (self._tanh(x)**2)
        elif self.func == 'reLU':
            if x > 0:
                return 1
            return 0
        elif self.func == 'leaky_reLU':
            if x > 0:
                return 1
            return self.leaky_val
        else:
            raise ValueError("The only available activation functions are: linear, sig, tanh, reLU and leaky_reLU.")
    
    def calculate(self, x):
        if self.func == 'linear':
            return self._linear(x)
        elif self.func == 'sig':
            return self._sigmoid(x)
        elif self.func == 'tanh':
            return self._tanh(x)
        elif self.func == 'reLU':
            return self._reLU(x)
        elif self.func == 'leaky_reLU':
            return self._leakyReLU(x)
        else:
            raise ValueError("The only available activation functions are: linear, sig, tanh, reLU and leaky_reLU.")
        


In [3]:
class LossFunction:

    def __init__(self, func='mse'):
        self.func = func

    def calculate(self, y_pred, y_true):
        if(self.func == 'mse'):
            return self._MSE(y_pred, y_true)
        elif (self.func == 'mae'):
            return self._MAE(y_pred, y_true)
        elif (self.func == 'binary_ce'):
            return self._binary_CE(y_pred, y_true)
        else:
            raise Exception("only supported loss function options are: mse, mae and binary_ce")
            

    def gradient(self, y_pred, y_true):
        if (self.func == 'mse'):
            return (2 / y_pred.size) * (y_pred - y_true)
        elif (self.func == 'mae'):
            return np.sign(y_pred - y_true)
        elif(self.func == 'binary_ce'):
            return (y_pred - y_true)/(y_pred*(1-y_pred))
            
        
    def _MSE(self, y_pred, y_true):
        return np.mean((y_true - y_pred)**2)

    def _MAE(self, y_pred, y_true):
        return np.mean(abs(y_true - y_pred))

    def _binary_CE(self, y_pred, y_true):
        epsilon = 1e-12
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        
        ce = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        return ce

In [4]:
loss = LossFunction('mae')
x = loss.calculate(np.array([1,0.5]), np.array([5,7]))
print(x)
x = loss.gradient(np.array([10,12,11]), np.array([12,1,115]))
print(x)

5.25
[-1  1 -1]


In [5]:
x = ActivationFunction('linear')
print(x.calculate(-10))

-10


In [6]:
class Neuron:
    
    def __init__(self, input_amt, func, leaky_val=0.01, learning_rate=0.01):
        self.weights = np.random.randn(input_amt) * 0.01
        self.bias = 0
        self.delta = 0
        self.z = 0
        self.activation_output_z = 0
        self.last_inputs = np.ones(input_amt)
        self.activation_func = ActivationFunction(func=func, leaky_val=leaky_val)
        self.learning_rate=learning_rate

    def forward(self, inputs):
        self.last_inputs = inputs.copy()
        self.z = np.dot(inputs, self.weights) + self.bias
        self.activation_output_z = self.activation_func.calculate(self.z)

        
        return self.activation_output_z

    def get_z(self):
        return self.z
        
    def get_delta(self):
        return self.delta

    def update_parameters(self, delta):
        self.delta = delta
        grad = self.delta * self.last_inputs
        # Clip gradients
        grad = np.clip(grad, -1.0, 1.0)
    
        self.weights -= self.learning_rate * grad
        self.bias -= self.learning_rate * self.delta
        

In [7]:
n = Neuron(input_amt=2,func='reLU')
print(n.forward([1,-2]))

0.006199278941855247


In [86]:
class Network:
    
    def __init__(self, network_shape:tuple, activation_funct="sig", leaky_val=0.01, loss="mse", learning_rate=0.01):

        self.shape = list(network_shape)
        self.network = []
        self.loss_func = LossFunction(loss)
        self.input_size = network_shape[0] # the input amount -----------
        self.activation_func = ActivationFunction(func=activation_funct, leaky_val=leaky_val)
        
        for i in range(1, len(network_shape)): #Append every other layer
            self.network.append([Neuron(network_shape[i-1], func=activation_funct,leaky_val=leaky_val, learning_rate=learning_rate) for _ in range(network_shape[i])])

    def fit(self, x, y, epochs=100):
        
        for epoch in range(epochs):
            total_loss = 0
            for feature, target in zip(x,y):
                predictions = self._forward_pass(feature)                
                self._back_propogation(predictions, target)

                loss = self.loss_func.calculate(predictions, target)
                total_loss += loss
                
            print(f"At Epoch {epoch+1}, Loss: {total_loss}")

    def predict(self, x):
        # x is expected to be 2D: (n_samples, n_features)
        predictions = np.zeros((x.shape[0], 1))  # Pre-allocate
        
        for i in range(x.shape[0]):
            predictions[i] = self._forward_pass(x[i])  # Process one sample
        
        return predictions
        
    def _forward_pass(self, inputs: np.ndarray):
        if len(inputs) != self.input_size:
            raise Exception(f"Err. Input size must be{self.input_size} as set earlier.")

        self.input_layer = inputs
        current = inputs.copy()
        for i in range(len(self.network)): #for each layer ------------------------

            returned_vals = []
            for n in self.network[i]: #Calculate the values returned by each neuron at the given layer ---
                returned_vals.append(n.forward(current))

            current = np.array(returned_vals)
        
        return current

    def _back_propogation(self, predicted_vals, true_vals):
        true_vals = np.atleast_1d(true_vals)
    
        loss_gradient = self.loss_func.gradient(predicted_vals, true_vals)
    
        for i in range(len(self.network) - 1, -1, -1):
            prev_loss_gradient = None
            if i > 0:
                prev_loss_gradient = np.zeros(len(self.network[i - 1]))
    
            for j, n in enumerate(self.network[i]):
                z = n.get_z()
    
                # Output layer delta
                delta = loss_gradient[j] * n.activation_func.gradient(z)
    
                old_weights = n.weights.copy()
                n.update_parameters(delta)
    
                if i > 0:
                    for k in range(len(old_weights)):
                        prev_loss_gradient[k] += delta * old_weights[k]
    
            if i > 0:
                loss_gradient = prev_loss_gradient


                
        
    

In [57]:
#Creating a linear dataset to test my NN -------------

from sklearn.datasets import make_regression


X, y = make_regression(
    n_samples=1000,      # Number of samples
    n_features=8,       # Number of features
    n_informative=6,    # Number of useful features
    n_targets=1,        # Number of regression targets
    bias=5,          # Bias term (intercept) in the linear model
    noise=0,         # Standard deviation of the Gaussian noise
    random_state=42     # Set a random state for reproducibility
)

In [58]:
type(X)

numpy.ndarray

In [75]:
nn = Network((8,1), activation_funct="linear", loss="mse", learning_rate=0.01)

In [76]:
from sklearn.model_selection import train_test_split

In [77]:
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [78]:
nn.fit(X_train, y_train)

At Epoch 1, Loss: 16460577.094997358
At Epoch 2, Loss: 15343175.963890633
At Epoch 3, Loss: 14262890.26476725
At Epoch 4, Loss: 13222147.510088881
At Epoch 5, Loss: 12221299.720803486
At Epoch 6, Loss: 11258880.88232293
At Epoch 7, Loss: 10334725.21661411
At Epoch 8, Loss: 9448465.130167376
At Epoch 9, Loss: 8600591.71104828
At Epoch 10, Loss: 7792333.739946226
At Epoch 11, Loss: 7022162.690675963
At Epoch 12, Loss: 6291720.833632362
At Epoch 13, Loss: 5601997.581365442
At Epoch 14, Loss: 4952615.956252799
At Epoch 15, Loss: 4343432.374258048
At Epoch 16, Loss: 3775101.810940007
At Epoch 17, Loss: 3247809.779949878
At Epoch 18, Loss: 2761534.44513606
At Epoch 19, Loss: 2315532.3193123084
At Epoch 20, Loss: 1909610.3737759215
At Epoch 21, Loss: 1544659.6684357745
At Epoch 22, Loss: 1220647.7807645372
At Epoch 23, Loss: 936467.7986272187
At Epoch 24, Loss: 691356.0052310849
At Epoch 25, Loss: 484181.4360278323
At Epoch 26, Loss: 314668.37536607543
At Epoch 27, Loss: 182204.1904312262
At 

In [79]:
#Testing sigmoid --------------------------------------------------
from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=1000,
    n_features=2,
    n_redundant=0,
    n_informative=2,
    n_clusters_per_class=1,
    class_sep=2.0,
    random_state=42
)

X = (X - X.mean(axis=0)) / X.std(axis=0)


In [85]:
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
nn = Network((2,3,1), activation_funct="sig", loss="binary_ce", learning_rate=0.05)
nn.fit(X_train, y_train, epochs=200)

At Epoch 1, Loss: 553.7462276707521
At Epoch 2, Loss: 245.19974714121665
At Epoch 3, Loss: 67.06259831986853
At Epoch 4, Loss: 45.12030173229146
At Epoch 5, Loss: 38.086179690335406
At Epoch 6, Loss: 34.808960287127185
At Epoch 7, Loss: 32.98680115946654
At Epoch 8, Loss: 31.861520536432106
At Epoch 9, Loss: 31.115771261703497
At Epoch 10, Loss: 30.595529478706837
At Epoch 11, Loss: 30.218006826765784
At Epoch 12, Loss: 29.935256817541735
At Epoch 13, Loss: 29.71788337603662
At Epoch 14, Loss: 29.547035090916673
At Epoch 15, Loss: 29.41017472295972
At Epoch 16, Loss: 29.298709593459947
At Epoch 17, Loss: 29.206599357005974
At Epoch 18, Loss: 29.129504740642666
At Epoch 19, Loss: 29.06424916989017
At Epoch 20, Loss: 29.00846827605559
At Epoch 21, Loss: 28.960375925777466
At Epoch 22, Loss: 28.91860457693947
At Epoch 23, Loss: 28.882094229447794
At Epoch 24, Loss: 28.850013847792518
At Epoch 25, Loss: 28.821704906502724
At Epoch 26, Loss: 28.79664027142395
At Epoch 27, Loss: 28.774393878